# First Julia Notebook
This is an exercise notebook. If you can see this notebook and run the cells it means you have setup Julia correctly. Try to run the following cell.

In [3]:
println("Hello World")

Hello World
